# Blockchain analysis
todo

## Basic setup

Here we will import the `pyspark` module and set up a `SparkSession`.  By default, we'll use a `SparkSession` running locally, with one Spark executor; we're dealing with small data, so it doesn't make sense to run against a cluster.


In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import regexp_replace

spark = SparkSession.builder.master("local[1]").getOrCreate()
sc = spark.sparkContext

## Loading the data

To obtain the graph representing the transaction in the Bitcoin network, we need to load set of nodes representing the wallets (fingerprints of the public keys) and the set of edges representing each transaction. For this example we will use two parquet files that were generated from the blockchain data by this [convertor](https://github.com/Jiri-Kremser/bitcoin-insights/tree/master/parquet-converter).

In [2]:
raw_nodes = spark.read.load("/data/nodes.parquet") \
                      .withColumnRenamed("_1", "id") \
                      .withColumnRenamed("_2", "Wallet")
raw_nodes.show(5)

+---+--------------------+
| id|              Wallet|
+---+--------------------+
|  0|bitcoinaddress_93...|
|  1|bitcoinaddress_4D...|
|  2|bitcoinaddress_BE...|
|  3|bitcoinaddress_4B...|
|  4|bitcoinaddress_44...|
+---+--------------------+
only showing top 5 rows



As you can see, each record in the wallet column contains a string `bitcoinaddress_<hash>`, where the hash is the actual address of the wallet. Let's remove the redundant prefix.

In [3]:
nodes = raw_nodes.withColumn("Wallet", regexp_replace("Wallet", "bitcoinaddress_", "")).cache()
nodes.show(5)

+---+--------------------+
| id|              Wallet|
+---+--------------------+
|  0|9303DBB4C75A56057...|
|  1|4D3826A813A4B4E9B...|
|  2|BECC6154EEF33464E...|
|  3|4B5E0300F11C2932F...|
|  4|44730B80C9D5EF65D...|
+---+--------------------+
only showing top 5 rows



As you can see, each record in the wallet column contains a string `bitcoinaddress_<hash>`, where the hash is the actual address of the wallet. Let's remove the redundant prefix.

We can also verify, that these addresses are real on https://blockchain.info/address/. 

Example:
 * get random address

In [4]:
random_address = nodes.rdd.takeSample(False, 1)[0][1]
random_address

u'BDBAC450149BEDEDDFAF968166B46E84839178FF'

 * create the link from the address: https://blockchain.info/address/{{random_address}} 
 
 (todo: http://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/python-markdown/readme.html)

In [5]:
edges = spark.read.load("/data/edges.parquet") \
                  .withColumnRenamed("srcId", "src") \
                  .withColumnRenamed("dstId", "dst") \
                  .cache()
edges.show(5)
edges.count()

+------+------+-----+
|   src|   dst| attr|
+------+------+-----+
|150102|107378|input|
|470403|107378|input|
|232249| 97703|input|
|539070| 97703|input|
|131174|176711|input|
+------+------+-----+
only showing top 5 rows



2087249

## Constructing the graph representation

Spark contains API for graph processing. It's called [graphx](https://spark.apache.org/graphx/) and it also comes with multiple built-in algorithms like page-rank. It uses the [Pregel API](https://spark.apache.org/docs/latest/graphx-programming-guide.html#pregel-api).

In [6]:
from graphframes import *

g = GraphFrame(nodes, edges).cache()

#### Get the top 10 wallets with respect to the transaction count

First, by sorting the nodes by `inDegree` which corresponds to the number of transactions received.

In [7]:
vertexInDegrees = g.inDegrees
vertexInDegrees.join(nodes, vertexInDegrees.id == nodes.id) \
               .drop("id") \
               .orderBy("inDegree", ascending=False) \
               .take(10)

[Row(inDegree=121885, Wallet=u'F4B004C3CA2E7F96F9FC5BCA767708967AF67A44'),
 Row(inDegree=69445, Wallet=u'E1BB16A26D591FD766C1B23FAEC067301AFA8A07'),
 Row(inDegree=68626, Wallet=u'5605C6DC8A9672A014225BAC565DB25BCEC649A2'),
 Row(inDegree=5763, Wallet=u'66A731E9FEB460A3A48B5C56BB635B3A409DBD56'),
 Row(inDegree=3100, Wallet=u'17EBC6064CB035D84DC177CE763D2C93FD5556C6'),
 Row(inDegree=2380, Wallet=u'7B1DD94268DF8E11BA27AB1C99C61E914C717246'),
 Row(inDegree=2338, Wallet=u'A84458A0E8F009B3780CAC779873D298CF22BE8A'),
 Row(inDegree=2113, Wallet=u'EB9A27CF65B32AABA6FD08D7C4DE65F4C7CF9361'),
 Row(inDegree=2080, Wallet=u'EFB0CD7206CCF14275643097970DB8A38497B61D'),
 Row(inDegree=2002, Wallet=u'9B3A7C3A61712270055C1E4AC6FF2704D3ACB1E0')]

Then by using the `outDegree` ~ # transactions sent

In [8]:
vertexOutDegrees = g.outDegrees
vertexOutDegrees.join(nodes, vertexOutDegrees.id == nodes.id) \
                .drop("id") \
                .orderBy("outDegree", ascending=False) \
                .take(10)

[Row(outDegree=2870, Wallet=u'7B1DD94268DF8E11BA27AB1C99C61E914C717246'),
 Row(outDegree=1965, Wallet=u'CBD3148D93C205F862599C080556A2A531146A3B'),
 Row(outDegree=1952, Wallet=u'9B3A7C3A61712270055C1E4AC6FF2704D3ACB1E0'),
 Row(outDegree=1694, Wallet=u'580740AC5A1B84C24D7EDB3F3AFC635BDFFB587B'),
 Row(outDegree=1675, Wallet=u'826AC9812C2BE5FE349A61ACD393F1F38E8C7D2D'),
 Row(outDegree=1590, Wallet=u'17EBC6064CB035D84DC177CE763D2C93FD5556C6'),
 Row(outDegree=1577, Wallet=u'EFB0CD7206CCF14275643097970DB8A38497B61D'),
 Row(outDegree=1568, Wallet=u'A84458A0E8F009B3780CAC779873D298CF22BE8A'),
 Row(outDegree=1559, Wallet=u'64B7D877B3DB608FD62EC8034770FD0AEFE9975A'),
 Row(outDegree=1559, Wallet=u'5445DAC9A29FCDE8C7C896CF063923FBED4F5D2C')]

You can verify on blockchain.info that the actual number of transaction is lower than what we have just calculated. This stems from the fact how Bitcoin works, when sending BTC from a wallet to another one, it actually sends all the BTC and the receiving node will sends back the rest. We are oversimplyfying here and you can find the details [here](https://en.bitcoin.it/wiki/Transaction) or [here](https://bitcoin.stackexchange.com/questions/9007/why-are-there-two-transaction-outputs-when-sending-to-one-address).

#### Find circles of length 2

In [9]:
motifs = g.find("(a)-[]->(b); (b)-[]->(a)")
# motifs.count()
motifs.show(5)

+--------------------+--------------------+
|                   a|                   b|
+--------------------+--------------------+
|[372848,C27A8B1D3...|[9015,F4F92E73273...|
|[372848,C27A8B1D3...|[9015,F4F92E73273...|
|[481075,AC3A9CBF2...|[16851,225B70F51E...|
|[481075,AC3A9CBF2...|[16851,225B70F51E...|
|[481075,AC3A9CBF2...|[16851,225B70F51E...|
+--------------------+--------------------+
only showing top 5 rows



#### Foo

In [10]:
# this fails with OOM error
#results = g.pageRank(resetProbability=0.15, maxIter=1)
#results.vertices.select("id", "pagerank").show()
#results.edges.select("src", "dst", "weight").show()

# g.labelPropagation(maxIter)

# this would be nice display(ranks.vertices.orderBy(ranks.vertices.pagerank.desc()).limit(20))

## Visualization of a sub-graph

Our data contain a lot of transactions (2 087 249 transactions among 546 651 wallets) so let's show only a small fraction of the transaction graph.

In [11]:
%%javascript
require.config({
    paths: {
        sigmajs: 'https://cdnjs.cloudflare.com/ajax/libs/sigma.js/1.2.0/sigma.min'
    }
});

require(['sigmajs']);

<IPython.core.display.Javascript object>

In [ ]:
# sub_nodes = g.vertices.filter("id < 10")
# sub_graph = GraphFrame(sub_nodes, edges)
# sub_graph



from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random


random.seed(42)

n_nodes = 40
n_edges = 200

graph_data = { 'nodes': [], 'edges': [] }

for i in range(n_nodes):
    graph_data['nodes'].append({
            "id": "n" + str(i),
            "label": "n" + str(i),
            "x": random.uniform(0,1),
            "y": random.uniform(0,1),
            "size": random.uniform(0.2,1)
        })

for j in range(n_edges):
    x_center = random.uniform(0,1)
    y_center = random.uniform(0,1)
    x_dist = random.uniform(0.1,0.5)
    y_dist = random.uniform(0.2,0.5)
    neighborhood = []
    for node in graph_data['nodes']:
        if abs(node['x'] - x_center) < x_dist:
            if abs(node['y'] - y_center) < y_dist:
                neighborhood.append(int(node['id'].replace('n','')))
    if len(neighborhood) >= 2:
        ends = random.sample(neighborhood,2)
        graph_data['edges'].append({
                "id": "e" + str(j),
                "source": "n" + str(ends[0]),
                "target": "n" + str(ends[1])
            })
        
pd.DataFrame(graph_data['nodes']).head()

pd.DataFrame(graph_data['edges']).head()

js_text_template = Template(open('js/sigma-graph.js','r').read())

js_text = js_text_template.substitute({'graph_data': json.dumps(graph_data),
                                       'container': 'graph-div'})

html_template = Template('''
<div id="graph-div" style="height:400px"></div>
<script> $js_text </script>
''')

HTML(html_template.substitute({'js_text': js_text}))